In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from sklearn.metrics import f1_score
from collections import Counter

!python3 -m nltk.downloader -d /Users/moiz/nltk_data punkt
nltk.data.path.append('/Users/moiz/nltk_data')

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read().strip()

def calculate_bleu(reference, candidate):
    reference_tokens = [nltk.word_tokenize(reference)]
    candidate_tokens = nltk.word_tokenize(candidate)
    smooth = SmoothingFunction().method1
    return sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=smooth)

def calculate_meteor(reference, candidate):
    reference_tokens = nltk.word_tokenize(reference)
    candidate_tokens = nltk.word_tokenize(candidate)
    return meteor_score([reference_tokens], candidate_tokens)

def calculate_rouge(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)[0]
    return scores

def calculate_f1(reference, candidate):
    reference_tokens = nltk.word_tokenize(reference)
    candidate_tokens = nltk.word_tokenize(candidate)
    common = Counter(reference_tokens) & Counter(candidate_tokens)
    true_positives = sum(common.values())
    pred_positives = len(candidate_tokens)
    real_positives = len(reference_tokens)
    precision = true_positives / pred_positives if pred_positives > 0 else 0
    recall = true_positives / real_positives if real_positives > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1

# Main Data

In [ ]:
path_to_candidateMain = 'candidateMain.txt' 

reference_textMain_Mistral = "Daraz's refund policy allows for returns or refunds on items that are damaged, defective, incorrect, or incomplete. However, for device-related issues after usage or beyond the return policy period, it is recommended to check if the item is covered under Seller or Brand Warranty. Items that are non-returnable include any custom-made items. For Buy More, Save More orders, partial cancellations are not permitted, but partial returns are allowed. This means that if you wish to return certain items from a Buy More, Save More order, you can do so, but you would need to cancel the entire order if you want to return one or two specific products. After the returned items have been quality checked by Daraz, a partial refund will be issued to the customer on an item-level basis for each bundle. To submit a return request, customers must do so within 14 days of receiving their order. Decisions on returns and refunds will be taken by Daraz in its sole discretion. Customers attempting to circumvent the terms and conditions and abuse subsidies by creating new accounts will be unable to place their orders."
reference_textMain_Phi = "Thank you for your email. We appreciate your inquiry. Daraz has a clear cancellation and return policy for Buy More, Save More orders. Full refunds are only provided for damaged, defective, incorrect, or incomplete items. If the item is a device-related issue after the return policy period or not covered under the Seller or Brand Warranty, the refund may not be provided. Regarding partial cancellations, Daraz does not allow it. However, customers have the option to return certain items out of their total Buy More, Save More order. If a certain number of products from the order are eligible for a return, a partial refund will be issued after the items have been returned by the customer and checked by Daraz. In case you are not satisfied with your purchase, you can submit a return request within 14 days from the delivery date. Daraz makes the final decision on returns and refunds, and they will be provided in their sole discretion. Customers are not allowed to circumvent the terms and conditions and abuse subsidies by creating new accounts."
reference_textMain_Qwen = ""
reference_textMain_Zephyr = ""
candidate_textMain = read_file(path_to_candidateMain)

bleu_scoreMain = calculate_bleu(reference_textMain, candidate_textMain)
meteor_scoreMain = calculate_meteor(reference_textMain, candidate_textMain)
rouge_scoresMain = calculate_rouge(reference_textMain, candidate_textMain)
f1_scoreMain = calculate_f1(reference_textMain, candidate_textMain)

print("BLEU Score (Main Data):", bleu_scoreMain)
print("METEOR Score (Main Data):", meteor_scoreMain)
print("ROUGE Scores (Main Data):", rouge_scoresMain)
print("F1 Score (Main Data):", f1_scoreMain)

# Seller

In [ ]:
path_to_referenceSeller = 'referenceSeller.txt'  
path_to_candidateSeller = 'candidateSeller.txt' 

reference_textSeller = read_file(path_to_referenceSeller)
candidate_textSeller = read_file(path_to_candidateSeller)

bleu_scoreSeller = calculate_bleu(reference_text, candidate_textSeller)
meteor_scoreSeller = calculate_meteor(reference_text, candidate_textSeller)
rouge_scoresSeller = calculate_rouge(reference_text, candidate_textSeller)
f1_scoreSeller = calculate_f1(reference_text, candidate_textSeller)

print("BLEU Score (Seller):", bleu_scoreSeller)
print("METEOR Score (Seller):", meteor_scoreSeller)
print("ROUGE Scores (Seller):", rouge_scoresSeller)
print("F1 Score (Seller):", f1_scoreSeller)